# Import Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import tensorflow as tf
from keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.config.experimental_run_functions_eagerly(True)

# Load the Images

In [ ]:
img_dir = '/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set'

# Image Preprocessing

In [ ]:
BATCH_SIZE = 64
IMAGE_SIZE = 150
input_shape = (150,150,1)

In [ ]:
data_gen = ImageDataGenerator(rescale=1./255,
                              validation_split = 0.2)

In [ ]:
train_gen = data_gen.flow_from_directory(img_dir,
                                         target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                         batch_size=BATCH_SIZE,
                                         color_mode="grayscale",
                                         shuffle=True,
                                         class_mode="binary",
                                         subset="training")

In [ ]:
val_gen = data_gen.flow_from_directory(img_dir,
                                       target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                       batch_size=BATCH_SIZE,
                                       color_mode="grayscale",
                                       shuffle=False,
                                       class_mode="binary",
                                       subset="validation")

In [ ]:
labels = train_gen.class_indices
classes = list(labels.keys())

print(classes)

# Model Building

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D ,BatchNormalization, Flatten, Dropout
from keras.regularizers import l2

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3, 3), activation="relu", input_shape=(150, 150, 1), kernel_regularizer=l2(0.01)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation="relu", kernel_regularizer=l2(0.01)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation="relu", kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_gen,
                    epochs=20,
                    validation_data=val_gen,
                    steps_per_epoch=3681 // BATCH_SIZE,
                    validation_steps=919 // BATCH_SIZE)

In [ ]:
test_loss, test_acc   = model.evaluate(val_gen)

# Prediction

I have taken a random image from dataset.

In [ ]:
from matplotlib.pyplot import imshow
from PIL import Image, ImageOps
from IPython.display import display


data = np.ndarray(shape=(1,150, 150, 1), dtype=np.float32)
image = Image.open(f'/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set/Brain Tumor/Cancer (1005).jpg')
size = (150, 150)
image = ImageOps.grayscale(image)
image = ImageOps.fit(image, size, Image.LANCZOS)
image_array = np.asarray(image)
display(image)
#normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
data = image_array.reshape((-1,150,150,1))
#data[0] = normalized_image_array

1 - Not Cancer

0 - Cancer

In [ ]:
prediction = model.predict(data)
print(prediction[0][0])